In [ ]:
using Cubature, PyPlot, DelimitedFiles

In [ ]:
function xi(eta,u)
    q = clamp((1.0+eta)/(1.0-eta),-1e80,1e80)
    return (log(q)+u*1im)/(eta+tan(u/2)*1im)    
end

function phi_pt1(r,t)
    eta = r/t
    q = clamp((1.0+eta)/(1.0-eta),-1e80,1e80)
    return exp(-t)/(4*pi*r*t^2)*t*log(q)
end

function integrand_pt(u,eta,t)
    return sec(u/2)^2*real((eta+1im*tan(u/2))*xi(eta,u)^3*exp(t/2*(1-eta^2)*xi(eta,u)))
end

function phi_pt(r,t)
    r = clamp(r,1e-10,1e80)
    eta = r/t
    g = 0.0
    if eta<1.0
        g,error = hquadrature(x->integrand_pt(x,eta,t),0,pi,abstol = 1e-2)
    end
    f = 1/(2*pi)*exp(-t) /(4*pi*r*t^2)*(t/2)^2*(1-eta^2)*g
    f = f + phi_pt1(r,t)
    return f
end

function phi_l(t,r)
    eta = r/t
    if eta<1.0
        f,error = hquadrature(w->phi_pt(t*sqrt(eta^2+w^2),t),0,sqrt(1-eta^2),abstol = 1e-5)
        phi_l0 = exp(-t)/(2*pi*t^2) / sqrt(1-eta^2)
        return phi_l0 + (2*t)*f
    else
        return 0.0
    end
end

In [ ]:
function integrand_B(z,tau,t,x,a,sigmas)
    d = (x .- a.*(t .- tau)) ./ (t .- tau)

    factor1 = sigmas.*tau .* exp.(-sigmas*(t .- tau))
    factor2 = 1.0 ./ sqrt.(z .* (1.0 .- d.^2))
    factor3 = 1.0 ./ (1.0 .- d.^2)
    factor4 = phi_pt.(tau,tau .* (z .+ d.^2 .* (1.0 .- z)))
    
    return factor1.*factor2.*factor3.*factor4
end

In [ ]:
t = 1.0
x = 0.3
a = 1.0
sigmas = 1.0
sigmat = sigmas
@time hquadrature(eta -> exp(-sigmat*eta)*phi_l(t-eta,abs(x-a*eta)),0,t ,reltol = 1e-2)

In [ ]:
@time hcubature(dummy-> integrand_B(dummy[1],dummy[2],t,x,a,sigmas),[0,0],[1,t-x/(1+a)],reltol = 1e-2)